In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_course_30.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 40 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(40): SRR4317611 SRR1636594 ... SRR7817645 SRR7817642
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 40
                   SRR4317611 SRR1636594 SRR7817673 SRR4317609 SRR7817613
ENSMUSG00000000001  15.053087  16.047024  15.566396  15.201088  15.669339
ENSMUSG00000000028   8.433167   7.826188   8.164691   8.349515   8.002616
ENSMUSG00000000049  20.584907  20.709859  20.120848  20.629986  20.508462
ENSMUSG00000000058   9.212261   9.429295   8.710940   9.507016   8.522645
ENSMUSG00000000085  11.094044  10.199346  10.447924   9.984149  10.578423
ENSMUSG00000000088  14.583323  16.232460  16.347877  14.585453  16.858858
                   SRR7817696 SRR4317610 SRR7817674 SRR3593527 SRR7817625
ENSMUSG00000000001  15.874948  15.343855  15.672890  15.018218  15.795126
ENSMUSG00000000028   7.363307   8.107192   7.507445   8.193708   7.617408
ENSMUSG00000000049  20.106274  20.639563  20.133894  20.952922  20.345028
ENSMUSG00000000058   8.613762   9.807282   9.023300   8.374296   8.451547
ENSMUSG00000000085  11.144845  10.456363  10.090989  10.764372  10.881871
ENSMUSG00000000088  16.763276  

In [7]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_course_30.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30 30 30 30 30
[26] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_course_30.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

[1] "ENSMUSG00000024122" "ENSMUSG00000036526"

$`R-MMU-202437`
[1] "ENSMUSG00000024122" "ENSMUSG00000026778" "ENSMUSG00000036526"

$`R-MMU-202443`
[1] "ENSMUSG00000024122" "ENSMUSG00000036526"

$`R-MMU-202459`
[1] "ENSMUSG00000024122" "ENSMUSG00000028191" "ENSMUSG00000036526"
[4] "ENSMUSG00000041135"

$`R-MMU-202466`
[1] "ENSMUSG00000024122" "ENSMUSG00000028191" "ENSMUSG00000036526"

$`R-MMU-202472`
[1] "ENSMUSG00000024122" "ENSMUSG00000027164" "ENSMUSG00000028191"
[4] "ENSMUSG00000032688" "ENSMUSG00000036526"

$`R-MMU-202478`
[1] "ENSMUSG00000024122" "ENSMUSG00000028191" "ENSMUSG00000032688"
[4] "ENSMUSG00000036526"

$`R-MMU-202489`
[1] "ENSMUSG00000024122" "ENSMUSG00000028191" "ENSMUSG00000036526"

$`R-MMU-2219524`
[1] "ENSMUSG00000024122"

$`R-MMU-2316429`
[1] "ENSMUSG00000024122"

$`R-MMU-9632858`
[1] "ENSMUSG00000024122" "ENSMUSG00000029053"

$`R-MMU-9632906`
[1] "ENSMUSG00000024122" "ENSMUSG00000025499" "ENSMUSG00000029053"
[4] "ENSMUSG00000030265"

$`R-MMU-9632910`
[1] "ENSMUSG0

[1] "ENSMUSG00000024241" "ENSMUSG00000040751" "ENSMUSG00000042626"
[4] "ENSMUSG00000059923"

$`R-MMU-2730842`
[1] "ENSMUSG00000024241" "ENSMUSG00000027665" "ENSMUSG00000031834"
[4] "ENSMUSG00000032462" "ENSMUSG00000040751" "ENSMUSG00000041417"
[7] "ENSMUSG00000042626" "ENSMUSG00000059923"

$`R-MMU-2730844`
[1] "ENSMUSG00000024241" "ENSMUSG00000042626" "ENSMUSG00000059923"

$`R-MMU-2730870`
[1] "ENSMUSG00000024241" "ENSMUSG00000027665" "ENSMUSG00000031834"
[4] "ENSMUSG00000032462" "ENSMUSG00000040751" "ENSMUSG00000041417"
[7] "ENSMUSG00000042626" "ENSMUSG00000059923"

$`R-MMU-5686073`
[1] "ENSMUSG00000024241" "ENSMUSG00000042626" "ENSMUSG00000059923"

$`R-MMU-5686074`
[1] "ENSMUSG00000024241" "ENSMUSG00000042626" "ENSMUSG00000059923"

$`R-MMU-5686315`
[1] "ENSMUSG00000024241" "ENSMUSG00000038894" "ENSMUSG00000059923"

$`R-MMU-74736`
[1] "ENSMUSG00000024241" "ENSMUSG00000038894" "ENSMUSG00000059923"

$`R-MMU-74746`
[1] "ENSMUSG00000024241" "ENSMUSG00000042626" "ENSMUSG00000059923"

$`R-M

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



$`R-MMU-1564120`
[1] "ENSMUSG00000025650" "ENSMUSG00000031740" "ENSMUSG00000043089"

$`R-MMU-2213195`
[1] "ENSMUSG00000025650"

$`R-MMU-2214324`
[1] "ENSMUSG00000025650"

$`R-MMU-5694427`
[1] "ENSMUSG00000025650" "ENSMUSG00000056050"

$`R-MMU-8944255`
[1] "ENSMUSG00000025650"

$`R-NUL-4084497`
[1] "ENSMUSG00000025650"

$`R-MMU-9018858`
[1] "ENSMUSG00000025701"

$`R-MMU-9018859`
[1] "ENSMUSG00000025701"

$`R-MMU-9018863`
[1] "ENSMUSG00000025701"

$`R-MMU-9018894`
[1] "ENSMUSG00000025701"

$`R-MMU-9020251`
[1] "ENSMUSG00000025701"

$`R-MMU-9020255`
[1] "ENSMUSG00000025701"

$`R-MMU-9020256`
[1] "ENSMUSG00000025701"

$`R-MMU-9020259`
[1] "ENSMUSG00000025701"

$`R-MMU-9020264`
[1] "ENSMUSG00000025701"

$`R-MMU-9020277`
[1] "ENSMUSG00000025701"

$`R-MMU-9020278`
[1] "ENSMUSG00000025701"

$`R-MMU-9020282`
[1] "ENSMUSG00000025701"

$`R-MMU-9024997`
[1] "ENSMUSG00000025701"

$`R-MMU-9025995`
[1] "ENSMUSG00000025701"

$`R-MMU-9025996`
[1] "ENSMUSG00000025701"

$`R-MMU-9025999`
[1] "ENSMUSG00000

 [1] "ENSMUSG00000060032" "ENSMUSG00000060081" "ENSMUSG00000061615"
 [4] "ENSMUSG00000061991" "ENSMUSG00000063021" "ENSMUSG00000063689"
 [7] "ENSMUSG00000063954" "ENSMUSG00000064220" "ENSMUSG00000068855"
[10] "ENSMUSG00000069270" "ENSMUSG00000069272" "ENSMUSG00000069301"
[13] "ENSMUSG00000069302" "ENSMUSG00000069309" "ENSMUSG00000071478"
[16] "ENSMUSG00000071516" "ENSMUSG00000078851" "ENSMUSG00000094248"
[19] "ENSMUSG00000094777" "ENSMUSG00000109324"

$`R-MMU-1855185`
[1] "ENSMUSG00000060733"

$`R-MMU-1855206`
[1] "ENSMUSG00000060733"

$`R-MMU-1855228`
[1] "ENSMUSG00000060733"

$`R-MMU-1855233`
[1] "ENSMUSG00000060733"

$`R-MMU-1449687`
[1] "ENSMUSG00000060807"

$`R-MMU-9757686`
[1] "ENSMUSG00000060807"

$`R-MMU-9757688`
[1] "ENSMUSG00000060807"

$`R-MMU-6814096`
[1] "ENSMUSG00000060904" "ENSMUSG00000063253"

$`R-MMU-5602295`
[1] "ENSMUSG00000061126"

$`R-MMU-9832937`
[1] "ENSMUSG00000061524" "ENSMUSG00000090125"

$`R-MMU-9832947`
[1] "ENSMUSG00000061524" "ENSMUSG00000090125"

$`R-MMU-

In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

[1] 9825
   [1] "R-MMU-170666"  "R-MMU-170671"  "R-MMU-170674"  "R-MMU-170686" 
   [5] "R-MMU-380073"  "R-MMU-392187"  "R-MMU-392195"  "R-MMU-392202" 
   [9] "R-MMU-749454"  "R-MMU-749456"  "R-MMU-751001"  "R-MMU-8964229"
  [13] "R-MMU-8964242" "R-MMU-8964329" "R-MMU-8982012" "R-MMU-8982019"
  [17] "R-MMU-8982020" "R-MMU-9021596" "R-MMU-9021600" "R-MMU-9021601"
  [21] "R-MMU-176298"  "R-MMU-176318"  "R-MMU-68914"   "R-MMU-68916"  
  [25] "R-MMU-68917"   "R-MMU-68960"   "R-MMU-481009"  "R-MMU-9021068"
  [29] "R-MMU-9021079" "R-MMU-3108203" "R-MMU-3108209" "R-MMU-3927824"
  [33] "R-MMU-4551655" "R-MMU-4551727" "R-MMU-4570463" "R-MMU-4570499"
  [37] "R-MMU-8937989" "R-MMU-163214"  "R-MMU-9709406" "R-MMU-9838627"
  [41] "R-MMU-9839113" "R-MMU-193641"  "R-MMU-193646"  "R-MMU-193650" 
  [45] "R-MMU-193653"  "R-MMU-193656"  "R-MMU-193665"  "R-MMU-193668" 
  [49] "R-MMU-193669"  "R-MMU-193679"  "R-MMU-193682"  "R-MMU-193684" 
  [53] "R-MMU-193694"  "R-MMU-193695"  "R-MMU-193696"  "R-MMU-193700

[1005] "R-MMU-3928633" "R-MMU-3928635" "R-MMU-3928639" "R-MMU-3928641"
[1009] "R-MMU-3928645" "R-MMU-3928657" "R-MMU-4093329" "R-MMU-4093331"
[1013] "R-MMU-4093332" "R-MMU-4093336" "R-MMU-4093339" "R-MMU-9696266"
[1017] "R-MMU-9696275" "R-MMU-9014741" "R-MMU-9014766" "R-MMU-9034539"
[1021] "R-MMU-196950"  "R-MMU-8952618" "R-MMU-8952620" "R-MMU-8955241"
[1025] "R-MMU-8955289" "R-MMU-8956040" "R-MMU-8956200" "R-MMU-983140" 
[1029] "R-MMU-983147"  "R-MMU-983156"  "R-MMU-983157"  "R-MMU-141423" 
[1033] "R-MMU-174057"  "R-MMU-174058"  "R-MMU-174070"  "R-MMU-174079" 
[1037] "R-MMU-174088"  "R-MMU-174104"  "R-MMU-174105"  "R-MMU-174119" 
[1041] "R-MMU-174120"  "R-MMU-174121"  "R-MMU-174132"  "R-MMU-174139" 
[1045] "R-MMU-174144"  "R-MMU-174157"  "R-MMU-174171"  "R-MMU-174195" 
[1049] "R-MMU-174202"  "R-MMU-174224"  "R-MMU-174227"  "R-MMU-174238" 
[1053] "R-MMU-174255"  "R-MMU-179410"  "R-MMU-179417"  "R-MMU-179421" 
[1057] "R-MMU-188191"  "R-MMU-188371"  "R-MMU-3788705" "R-MMU-3788708"
[1061]

[1465] "R-MMU-9691174" "R-MMU-9751201" "R-NUL-9033341" "R-MMU-111439" 
[1469] "R-MMU-158441"  "R-MMU-158447"  "R-MMU-158481"  "R-MMU-158484" 
[1473] "R-MMU-159046"  "R-MMU-159050"  "R-MMU-170796"  "R-MMU-191830" 
[1477] "R-MMU-2990882" "R-MMU-3000348" "R-MMU-3000399" "R-MMU-3000411"
[1481] "R-MMU-450387"  "R-MMU-450595"  "R-MMU-4551649" "R-MMU-4551679"
[1485] "R-MMU-4570493" "R-MMU-4615872" "R-MMU-4615987" "R-MMU-4655355"
[1489] "R-MMU-5228508" "R-MMU-5228523" "R-MMU-5252041" "R-MMU-5578744"
[1493] "R-MMU-75097"   "R-MMU-75098"   "R-MMU-77587"   "R-MMU-77594"  
[1497] "R-MMU-2028294" "R-MMU-2328033" "R-MMU-2395320" "R-MMU-2395328"
[1501] "R-MMU-70286"   "R-MMU-1296127" "R-MMU-352379"  "R-MMU-352385" 
[1505] "R-MMU-9018808" "R-MMU-8852128" "R-MMU-8852131" "R-MMU-8852132"
[1509] "R-MMU-8852133" "R-MMU-8852134" "R-MMU-8852136" "R-MMU-983153" 
[1513] "R-MMU-158118"  "R-MMU-158137"  "R-MMU-216072"  "R-MMU-265425" 
[1517] "R-MMU-354066"  "R-MMU-354073"  "R-MMU-354087"  "R-MMU-354124" 
[1521]

[1925] "R-MMU-430190"  "R-MMU-430200"  "R-MMU-2106614" "R-MMU-2106615"
[1929] "R-MMU-2106625" "R-MMU-210805"  "R-MMU-426464"  "R-MMU-9834821"
[1933] "R-MMU-8870499" "R-MMU-5694423" "R-MMU-5694425" "R-MMU-6809014"
[1937] "R-MMU-6809015" "R-MMU-8847637" "R-MMU-8847638" "R-MMU-9692809"
[1941] "R-MMU-5250947" "R-MMU-5683930" "R-MMU-9033499" "R-MMU-9033516"
[1945] "R-MMU-9033533" "R-MMU-6798317" "R-MMU-8849878" "R-MMU-8849906"
[1949] "R-MMU-9610750" "R-MMU-6811417" "R-MMU-6811418" "R-MMU-1247922"
[1953] "R-MMU-1980044" "R-MMU-2032379" "R-MMU-9013059" "R-MMU-9013284"
[1957] "R-MMU-8952630" "R-MMU-8952631" "R-MMU-6810464" "R-MMU-5685341"
[1961] "R-MMU-5685838" "R-MMU-5685985" "R-MMU-5685994" "R-MMU-5686398"
[1965] "R-MMU-5686440" "R-MMU-5693539" "R-MMU-5693584" "R-MMU-5693620"
[1969] "R-MMU-5681987" "R-MMU-5682377" "R-MMU-8869425" "R-MMU-68919"  
[1973] "R-MMU-68940"   "R-MMU-68944"   "R-MMU-69019"   "R-MMU-9749253"
[1977] "R-MMU-9757258" "R-MMU-444773"  "R-MMU-879674"  "R-MMU-2426496"
[1981]

[2385] "R-MMU-6785165" "R-MMU-8983003" "R-MMU-8983059" "R-MMU-8983063"
[2389] "R-MMU-9025969" "R-MMU-163296"  "R-MMU-163310"  "R-MMU-9838321"
[2393] "R-MMU-9630993" "R-MMU-6785126" "R-MMU-6785342" "R-MMU-6785361"
[2397] "R-MMU-6786155" "R-MMU-6786166" "R-MMU-6786171" "R-MMU-9835411"
[2401] "R-MMU-1112538" "R-MMU-1112565" "R-MMU-1112587" "R-MMU-1112602"
[2405] "R-MMU-1112604" "R-MMU-1112727" "R-MMU-1169142" "R-MMU-1169206"
[2409] "R-MMU-2671836" "R-MMU-2671854" "R-MMU-2671877" "R-MMU-2730597"
[2413] "R-MMU-2730598" "R-MMU-6784006" "R-MMU-6784323" "R-MMU-6784763"
[2417] "R-MMU-6784765" "R-MMU-6784791" "R-MMU-6790038" "R-MMU-8848087"
[2421] "R-MMU-8848124" "R-MMU-8875816" "R-MMU-8875817" "R-MMU-8950724"
[2425] "R-MMU-8950733" "R-MMU-8950780" "R-MMU-8950782" "R-MMU-8982162"
[2429] "R-MMU-8982163" "R-MMU-8982165" "R-MMU-8983371" "R-MMU-8983373"
[2433] "R-MMU-8983374" "R-MMU-8983378" "R-MMU-8983379" "R-MMU-8985900"
[2437] "R-MMU-8985929" "R-MMU-8985966" "R-MMU-8985981" "R-MMU-8985988"
[2441]

...

[5985] "R-MMU-2468040" "R-MMU-2468041" "R-MMU-2473151" "R-MMU-2545203"
[5989] "R-MMU-2545253" "R-MMU-437136"  "R-MMU-437139"  "R-MMU-9014641"
[5993] "R-MMU-8849042" "R-MMU-8865265" "R-MMU-9625589" "R-MMU-191299" 
[5997] "R-NUL-9624599" "R-NUL-9624615" "R-NUL-9624625" "R-MMU-8944236"
[6001] "R-MMU-9706293" "R-MMU-9706319" "R-NUL-5623510" "R-MMU-5663129"
[6005] "R-MMU-879764"  "R-MMU-8852317" "R-MMU-8852324" "R-MMU-8852331"
[6009] "R-MMU-8852337" "R-MMU-8852351" "R-MMU-9624893" "R-MMU-9007447"
[6013] "R-MMU-8853496" "R-MMU-8854052" "R-MMU-8939688" "R-MMU-8956099"
[6017] "R-MMU-9008677" "R-MMU-9708525" "R-MMU-389652"  "R-MMU-73800"  
[6021] "R-MMU-73828"   "R-MMU-209059"  "R-NUL-209065"  "R-NUL-209132" 
[6025] "R-MMU-9714477" "R-MMU-432946"  "R-MMU-8949668" "R-MMU-9010205"
[6029] "R-MMU-212642"  "R-MMU-352108"  "R-MMU-352136"  "R-MMU-381668" 
[6033] "R-MMU-381727"  "R-MMU-392834"  "R-MMU-939265"  "R-MMU-5686924"
[6037] "R-MMU-5687183" "R-MMU-5693533" "R-MMU-5693575" "R-MMU-5693615"
[6041]

[6445] "R-MMU-5683714" "R-MMU-379432"  "R-MMU-109338"  "R-MMU-109998" 
[6449] "R-MMU-165680"  "R-MMU-5653968" "R-MMU-5672010" "R-MMU-5672817"
[6453] "R-MMU-5675790" "R-MMU-9018836" "R-MMU-162742"  "R-MMU-162863" 
[6457] "R-MMU-5694431" "R-MMU-6783221" "R-MMU-173636"  "R-MMU-173739" 
[6461] "R-MMU-173740"  "R-MMU-173745"  "R-MMU-183122"  "R-MMU-183126" 
[6465] "R-MMU-183130"  "R-MMU-199518"  "R-MMU-444647"  "R-MMU-8851436"
[6469] "R-MMU-8852874" "R-MMU-8853252" "R-MMU-964811"  "R-MMU-977619" 
[6473] "R-MMU-981535"  "R-MMU-981539"  "R-MMU-482804"  "R-MMU-482812" 
[6477] "R-MMU-209148"  "R-NUL-209146"  "R-MMU-5675373" "R-MMU-5695957"
[6481] "R-MMU-1855158" "R-MMU-1855193" "R-MMU-1855194" "R-MMU-1855223"
[6485] "R-MMU-1855227" "R-MMU-9022717" "R-MMU-9022722" "R-MMU-192422" 
[6489] "R-MMU-192434"  "R-MMU-975593"  "R-MMU-6809287" "R-MMU-451634" 
[6493] "R-MMU-5684261" "R-MMU-8954117" "R-MMU-109813"  "R-MMU-109822" 
[6497] "R-MMU-109823"  "R-MMU-1250380" "R-MMU-1250383" "R-MMU-1306969"
[6501]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "R-MMU-1474146" "R-MMU-1474158" "R-MMU-1296037"
[8941] "R-MMU-6797627" "R-MMU-6786650" "R-MMU-449734"  "R-MMU-391940" 
[8945] "R-MMU-428625"  "R-MMU-888592"  "R-MMU-5362564" "R-MMU-71707"  
[8949] "R-MMU-174391"  "R-MMU-2408551" "R-MMU-8869606" "R-MMU-8869633"
[8953] "R-MMU-844619"  "R-MMU-877396"  "R-MMU-202703"  "R-MMU-391152" 
[8957] "R-MMU-391153"  "R-MMU-391157"  "R-MMU-391158"  "R-MMU-548973" 
[8961] "R-MMU-373071"  "R-MMU-2889070" "R-MMU-428205"  "R-MMU-5628905"
[8965] "R-MMU-168400"  "R-MMU-168405"  "R-MMU-622310"  "R-MMU-5251989"
[8969] "R-MMU-9714186" "R-MMU-3222259" "R-MMU-6805792" "R-MMU-375138" 
[8973] "R-MMU-9017868" "R-NUL-9013647" "R-NUL-9013728" "R-NUL-9014136"
[8977] "R-MMU-202713"  "R-MMU-6814797" "R-MMU-352107"  "R-MMU-5685605"
[8981] "R-MMU-9007813" "R-MMU-6804527" "R-MMU-209903"  "R-MMU-158893" 
[8985] "R-MMU-202726"  "R-MMU-9717432" "R-MMU-434989"  "R-MMU-9649963"
[8989] "R-MMU-6784959" "R-MMU-8985270" "R-MMU-1222685" "R-MMU-6801687"
[8993] "R-MMU-6787632" "R-MM

[9397] "R-MMU-5661126" "R-MMU-5423038" "R-MMU-6805740" "R-MMU-428585" 
[9401] "R-MMU-9844587" "R-MMU-1614544" "R-MMU-742345"  "R-MMU-417820" 
[9405] "R-MMU-9707606" "R-MMU-9707683" "R-MMU-9707856" "R-MMU-1483087"
[9409] "R-MMU-1483211" "R-MMU-1483219" "R-MMU-1483229" "R-MMU-5693691"
[9413] "R-MMU-9749647" "R-MMU-9749792" "R-MMU-5696119" "R-MMU-9673827"
[9417] "R-MMU-2872444" "R-MMU-204617"  "R-MMU-204647"  "R-MMU-204662" 
[9421] "R-MMU-1482932" "R-MMU-451360"  "R-MMU-5682285" "R-MMU-517536" 
[9425] "R-MMU-191983"  "R-MMU-2161660" "R-MMU-727740"  "R-MMU-727768" 
[9429] "R-MMU-1483096" "R-MMU-1483159" "R-MMU-417842"  "R-MMU-429786" 
[9433] "R-MMU-190782"  "R-MMU-191072"  "R-MMU-375340"  "R-MMU-174959" 
[9437] "R-MMU-174963"  "R-MMU-8851679" "R-MMU-444171"  "R-MMU-9831007"
[9441] "R-MMU-8869694" "R-MMU-2161651" "R-MMU-9729538" "R-MMU-912383" 
[9445] "R-NUL-1214205" "R-MMU-420724"  "R-MMU-9703895" "R-MMU-444100" 
[9449] "R-NUL-9622751" "R-NUL-9622814" "R-MMU-391937"  "R-MMU-418918" 
[9453]

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

 [1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40


In [12]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30 30 30 30 30
[26] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR4317611 SRR1636594 SRR7817673 SRR4317609 SRR7817613
ENSMUSG00000000001  15.053087  16.047024  15.566396  15.201088  15.669339
ENSMUSG00000000028   8.433167   7.826188   8.164691   8.349515   8.002616
ENSMUSG00000000049  20.584907  20.709859  20.120848  20.629986  20.508462
ENSMUSG00000000058   9.212261   9.429295   8.710940   9.507016   8.522645
ENSMUSG00000000085  11.094044  10.199346  10.447924   9.984149  10.578423
ENSMUSG00000000088  14.583323  16.232460  16.347877  14.585453  16.858858
ENSMUSG00000000120   8.489823   8.421934   8.817243   8.881583   9.892773
ENSMUSG00000000126   7.086750   7.346629   8.060751   7.318097   7.326955
ENSMUSG00000000127   9.805555   9.714354   9.648008   9.759318  10.511006
ENSMUSG00000000142  11.241016   9.813289   9.466599  10.773091   9.978160
ENSMUSG00000000149  16.090536  14.848586  15.962552  15.502433  16.047826
ENSMUSG00000000154  15.824893  15.582817  15.373021  15.551797  15.670571
ENSMUSG00000000168  14.063870  14.7256

ENSMUSG00000001105  13.465653  13.261919  13.287235  13.370577  13.569674
ENSMUSG00000001119   9.579709  10.238883   8.832220   9.417969   9.549497
ENSMUSG00000001123  18.480315  18.052118  17.386505  18.582072  18.719909
ENSMUSG00000001127  15.212285  15.060026  15.244411  15.051204  15.193197
ENSMUSG00000001128  13.323908  11.962055  13.612571  11.425497  11.683070
ENSMUSG00000001131   6.928900   6.971449   6.695739   6.767213   6.050532
ENSMUSG00000001143  11.598630  11.177559  11.314897  11.105627  11.764133
ENSMUSG00000001155  18.673540  18.403553  18.360259  18.814201  19.070718
ENSMUSG00000001158  10.811098  12.162075  11.273192  11.270194  11.264127
ENSMUSG00000001173  10.422609  10.859871  10.037906  10.497875  10.426331
ENSMUSG00000001175  16.324663  16.960286  17.306866  16.162534  17.500822
ENSMUSG00000001211  17.449972  16.196025  16.947643  17.016109  17.180918
ENSMUSG00000001225   6.064750   6.026101   6.701510   6.767213   7.002448
ENSMUSG00000001228   7.605400   7.2839

ENSMUSG00000002249  10.566455  10.241538  10.948512  10.305743  10.893062
ENSMUSG00000002250  10.689417   9.629775  10.599406  10.009898  13.873839
ENSMUSG00000002257   7.089603   7.248825   7.212466   6.623170   7.545117
ENSMUSG00000002279  13.599440  13.335560  13.272061  13.113524  13.538332
ENSMUSG00000002289  16.806881  16.092636  14.286646  16.577478  13.958044
ENSMUSG00000002297   8.569305   8.364206   8.786913   8.944431   8.958299
ENSMUSG00000002307  11.342912  10.668336  11.299752  11.107246  11.289127
ENSMUSG00000002308   9.293506   9.710786   8.809708   8.532500   9.438963
ENSMUSG00000002324   9.158934   8.584086   8.700024   8.332602   8.697630
ENSMUSG00000002325  13.972423  13.184794  14.380354  13.287251  13.635747
ENSMUSG00000002326  11.116076  11.816902  11.165609  11.153831  11.668144
ENSMUSG00000002346  16.477827  15.498472  15.241274  15.515188  15.955739
ENSMUSG00000002365  13.724006  14.153872  13.861899  13.258251  13.953065
ENSMUSG00000002379  13.770013  13.4078

ENSMUSG00000003379   8.398360   7.507177   6.064361   7.443901   7.131337
ENSMUSG00000003402  15.946853  15.906130  15.893910  15.818507  16.159089
ENSMUSG00000003411   6.906397   7.130471   7.498065   7.446211   7.326955
ENSMUSG00000003418   6.702986   6.555506   6.064361   7.010377   6.050532
ENSMUSG00000003421  11.616213  11.441479  12.155095  11.744115  12.613396
ENSMUSG00000003429  14.936119  15.233865  13.899215  15.380581  14.439512
ENSMUSG00000003435  14.913060  14.129050  15.019829  14.546980  14.768619
ENSMUSG00000003436   6.064750   6.026101   6.064361   6.081757   6.050532
ENSMUSG00000003437  12.222306  13.500591  13.104479  12.594918  13.050223
ENSMUSG00000003444   9.431037   8.812362   9.138810   9.497971   9.396173
ENSMUSG00000003452   7.219683   7.067632   6.064361   7.325983   6.050532
ENSMUSG00000003458  14.548037  13.640750  13.844142  14.285617  14.074847
ENSMUSG00000003464  16.627412  16.091209  16.144035  16.220407  15.859984
ENSMUSG00000003476   6.064750   6.8733

ENSMUSG00000004654   6.064750   6.026101   6.064361   6.767213   6.050532
ENSMUSG00000004655  16.010138  14.688067  12.957222  16.060793  14.701437
ENSMUSG00000004665   9.298883   9.338781   9.453226   8.702630   9.287499
ENSMUSG00000004667  12.103393  12.205224  12.354864  11.955110  12.467081
ENSMUSG00000004668   6.928900   6.026101   6.927029   6.081757   6.050532
ENSMUSG00000004677  12.997488  11.825702  13.038516  12.695426  13.179795
ENSMUSG00000004709   7.086750   6.555506   7.445535   7.325983   6.648960
ENSMUSG00000004730  12.809160  10.240925  12.645936  10.974229  10.591094
ENSMUSG00000004768   9.735816  11.297983  10.579201  10.457701  10.727115
ENSMUSG00000004771  13.518532  15.179310  14.298687  13.683082  14.660819
ENSMUSG00000004788  11.407268  11.200393  11.311240  11.033517  11.925091
ENSMUSG00000004789  17.199414  17.086566  16.864186  16.985029  17.300808
ENSMUSG00000004791   6.064750   6.026101   6.064361   6.081757   6.050532
ENSMUSG00000004815  13.469681  14.1231

...

ENSMUSG00000006373  19.998844  20.117302  19.953394  20.103416  20.172746
ENSMUSG00000006386  10.436771  11.404880  11.368925  11.250175  11.611599
ENSMUSG00000006390  13.815539  13.013839  13.905145  13.873177  13.890817
ENSMUSG00000006392  13.236827  12.744816  13.283277  12.608921  12.900348
ENSMUSG00000006398  10.259094   8.179168  10.795266   8.805379   8.579019
ENSMUSG00000006403   7.583984   8.301224   7.964695   7.709407   8.393686
ENSMUSG00000006411   6.992457   7.530975   7.399405   7.690593   7.855551
ENSMUSG00000006418  14.490083  13.498649  14.393024  14.267035  14.135530
ENSMUSG00000006423  12.578537  13.346210  12.437157  12.972543  12.931675
ENSMUSG00000006442  13.260390  11.547952  12.583560  12.647019  12.056493
ENSMUSG00000006445  12.132868  10.993954  11.459923  11.915510  10.904776
ENSMUSG00000006456  11.888829  11.020608  11.637104  11.331892  10.661294
ENSMUSG00000006457   6.844757   7.053448   7.052671   6.860180   6.586413
ENSMUSG00000006464   8.544096   8.4023

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   6.047305   6.005025   6.060645   6.052669   6.034162
ENSMUSG00000019906   9.079848   6.925178   7.646998   7.144606   7.298768
ENSMUSG00000019907  13.033543  13.213672  12.443312  13.013358  13.192309
ENSMUSG00000019916  11.770794  10.670828  10.609148  10.440619  10.406166
ENSMUSG00000019920  13.076300  13.062500  12.784762  12.682877  13.151604
ENSMUSG00000019923  15.636846  16.558304  16.082078  16.433092  16.616074
ENSMUSG00000019927  10.589915  10.134645  10.505217  10.901868  10.266901
ENSMUSG00000019929  16.855142  18.210682  17.603882  17.959355  17.937192
ENSMUSG00000019932   6.047305   6.005025   6.060645   6.052669   6.034162
ENSMUSG00000019935  11.602489   8.000951   9.361590   8.253408   7.944210
ENSMUSG00000019942  10.780510  10.112046  11.577618  10.301493  10.136549
ENSMUSG00000019943  13.326377  13.847507  12.613407  12.691376  13.083381
ENSMUSG00000019944  10.964866  12.211934  11.122321  11.876792  11.436080
ENSMUSG00000019947  11.901288  12.519477  11.703018  11.

ENSMUSG00000020232  12.104320  11.143772  12.167496  12.136370  12.006229
ENSMUSG00000020235  10.918515  10.573521  11.267612  11.424993  10.872494
ENSMUSG00000020241  11.346258  11.261560  11.165517  11.344718  11.223232
ENSMUSG00000020246  11.492243  11.372116  11.004639  11.552416  11.331026
ENSMUSG00000020248  11.976352  11.327748  11.606874  11.012284  11.341316
ENSMUSG00000020250  17.630397  18.089567  18.146267  18.016206  18.080379
ENSMUSG00000020256   7.531041   8.271066   7.770347   8.035202   8.014602
ENSMUSG00000020257  14.545655  13.415824  13.328255  13.460797  13.050171
ENSMUSG00000020258  16.829435  11.570921  12.778687  10.760020   9.838796
ENSMUSG00000020260  12.344770  12.071874  12.211120  12.615818  12.275655
ENSMUSG00000020261  12.095089  11.840172  11.769524  12.159406  11.317678
ENSMUSG00000020262   9.637509  11.189035  10.176935  10.865830  10.521285
ENSMUSG00000020264   7.634451   8.288857   7.645534   8.825865   7.736171
ENSMUSG00000020265  13.741513  14.1156

ENSMUSG00000020527  12.029928  11.353964  11.591857  11.136755  10.894176
ENSMUSG00000020532  14.703408  13.538170  13.618755  13.341919  13.401856
ENSMUSG00000020534  14.274818  11.302091  12.420037  11.835806  12.022913
ENSMUSG00000020537  13.530318  13.138827  13.467727  13.748514  13.764832
ENSMUSG00000020538  16.426047  12.279804  12.692760  13.551882  12.602884
ENSMUSG00000020542   6.047305   6.846070   6.687417   6.870711   6.034162
ENSMUSG00000020544  11.281445  10.210992  10.780897  10.854205  11.035432
ENSMUSG00000020553  14.144666  15.138701  14.535428  14.796610  13.806420
ENSMUSG00000020561  11.560117  10.965532  11.106460  11.601971  11.797460
ENSMUSG00000020566   6.467908   6.005025   6.274673   7.061397   7.202667
ENSMUSG00000020571  17.161419  16.274115  16.541386  16.905747  16.762965
ENSMUSG00000020572  15.182154  16.007138  15.105760  14.911208  15.238742
ENSMUSG00000020573   8.723472   9.151301   8.821323   8.836312   9.227305
ENSMUSG00000020576  13.167610  11.9712

In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

  11.643163
ENSMUSG00000020680  15.040488  16.404797  14.677450  16.014831  15.194906
ENSMUSG00000020681   7.511645   7.457571   7.659823   7.745509   7.801668
ENSMUSG00000020687  12.785274  13.008971  13.192341  13.436835  12.259076
ENSMUSG00000020689  10.064648   9.930553   9.015459  10.146560   9.253313
ENSMUSG00000020695   7.852450   7.555468   7.723329   8.115779   8.141382
ENSMUSG00000020696  13.873878  13.379286  13.555384  13.845259  13.319038
ENSMUSG00000020697  13.134892  12.627110  13.150190  12.783455  12.750070
ENSMUSG00000020698   7.861501   7.763791   7.483845   7.738486   6.991607
ENSMUSG00000020700  11.706941  10.517089  10.745512  11.278471  11.383971
ENSMUSG00000020702   6.064750   6.026101   6.646643   6.081757   6.050532
ENSMUSG00000020704   6.064750   6.026101   6.064361   6.081757   6.050532
ENSMUSG00000020705  13.987103  14.514249  13.932834  13.737009  13.815381
ENSMUSG00000020706  13.918740  13.938212  14.335943  13.742604  13.965311
ENSMUSG00000020708  14.884

ENSMUSG00000020986  15.933903  16.146666  15.434882  15.822824  15.376499
ENSMUSG00000020988  13.544144  15.187119  13.957452  14.177500  14.403947
ENSMUSG00000020993  10.960035  12.576216  11.798214  11.355851  12.328540
ENSMUSG00000020994  13.976578  15.877872  14.611890  14.114182  14.257885
ENSMUSG00000021000  17.523068  17.992554  17.642656  18.037564  17.557785
ENSMUSG00000021003   9.922874   9.454346   9.766745   9.380655   9.969955
ENSMUSG00000021013   9.205531   9.672460   8.703671   9.864946   9.930510
ENSMUSG00000021018   9.610601  10.762808   9.971199  10.018580  10.369860
ENSMUSG00000021024  16.096054  16.250329  15.462308  15.902767  15.959497
ENSMUSG00000021025  13.914918  13.682269  12.845138  12.951113  12.983456
ENSMUSG00000021027  10.439155  11.015537  11.175060  11.001841  10.581790
ENSMUSG00000021028   9.477946  10.023378   8.545058   9.430471   9.617923
ENSMUSG00000021032   7.331734   6.871016   6.064361   7.184236   6.050532
ENSMUSG00000021033  19.219794  19.3164

ENSMUSG00000021335  11.681636  11.121812  11.197928  11.578306  11.470204
ENSMUSG00000021338  13.207115  11.148627  13.708950  12.079931  12.969452
ENSMUSG00000021339  16.448500  17.216847  16.558598  16.420473  16.394076
ENSMUSG00000021340  18.053931  17.756643  17.795377  17.920254  17.656823
ENSMUSG00000021357  11.468204  11.246849  11.338925  10.916164  11.256711
ENSMUSG00000021359   6.064750   6.325806   6.064361   6.232406   6.050532
ENSMUSG00000021364  18.706332  18.792532  18.417752  18.767286  19.146990
ENSMUSG00000021367   7.843333   7.512209   6.064361   7.006399   7.410556
ENSMUSG00000021368  10.231623  10.419551   9.964464  10.850961  10.388539
ENSMUSG00000021374  13.340663  13.006610  13.035385  13.051236  12.748485
ENSMUSG00000021376  14.413531  13.939878  13.694904  14.408146  13.784543
ENSMUSG00000021377  12.522530  14.249788  13.330096  13.753750  13.401935
ENSMUSG00000021379   8.300310   8.318132   7.212466   9.000649   8.307617
ENSMUSG00000021385  10.899442  10.5968

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  10.673993   9.945244   9.657393
ENSMUSG00000024036   9.100608   8.015155  10.345889   8.730461   8.759080
ENSMUSG00000024038  15.188223  14.755095  13.631580  15.444060  15.013726
ENSMUSG00000024039  18.744641  17.930552  17.659841  18.666251  19.230696
ENSMUSG00000024043   7.753863   7.278799   7.329072   7.745509   7.857376
ENSMUSG00000024044   8.787634   8.795689   8.742576   8.245097   7.911729
ENSMUSG00000024052  18.123757  18.459625  16.974674  18.013002  17.264499
ENSMUSG00000024053   8.170097   8.023010   8.851777   8.394824   7.959934
ENSMUSG00000024056   7.217285   7.055720   6.701510   7.432271   6.860378
ENSMUSG00000024064   7.404621   6.026101   6.927029   7.446211   7.626060
ENSMUSG00000024065  14.721108  14.310517  11.971236  15.126964  14.732480
ENSMUSG00000024066  16.772686  16.913333  17.399293  16.561511  16.833750
ENSMUSG00000024068  13.284844  13.410178  13.231343  13.280838  13.263044
ENSMUSG00000024077  11.918346  12.032974  11.538055  12.069830  11.457095
ENSM

...

ENSMUSG00000021385  11.191162  10.530257  10.710810  11.057067  10.609678
ENSMUSG00000021390   8.917860   9.019438   8.799756   9.168789   9.008116
ENSMUSG00000021391   7.528323   7.419693   7.645534   7.321124   7.123984
ENSMUSG00000021408  12.892582  11.922408  12.117685  12.095522  11.987154
ENSMUSG00000021413  15.344156  15.502124  14.568597  14.849931  14.855187
ENSMUSG00000021417  14.936018  16.631157  16.253389  16.494954  15.870472
ENSMUSG00000021418   8.732596   8.957759   8.663158   9.150693   9.695065
ENSMUSG00000021423  10.088757  10.500864  11.034045  10.658283  10.353254
ENSMUSG00000021428  11.065152  10.630392  10.946525  11.171902  11.424566
ENSMUSG00000021431  12.386292  12.116424  11.600749  11.956595  12.232099
ENSMUSG00000021432  10.813795  11.427063  11.455049  11.795139  12.061099
ENSMUSG00000021448   6.047305   6.227915   6.060645   6.052669   6.034162
ENSMUSG00000021451  10.264112  10.320910  11.219146  10.510166  10.574941
ENSMUSG00000021456   8.918887   9.5717

ENSMUSG00000021725   9.477742   9.888828   9.214802   9.566993   9.418013
ENSMUSG00000021728  10.789500  11.895303  11.530823  12.680553  11.341000
ENSMUSG00000021730   6.047305   6.005025   6.060645   6.052669   6.034162
ENSMUSG00000021731  12.090910  11.227118  12.050129  12.363951  12.097313
ENSMUSG00000021732   6.047305   6.005025   6.060645   6.052669   6.034162
ENSMUSG00000021733  11.177033  11.876310  10.706267  11.141832  11.376669
ENSMUSG00000021737  15.057067  13.874064  13.979413  14.092527  14.613877
ENSMUSG00000021738  12.851215  13.023683  12.567528  12.855914  13.149814
ENSMUSG00000021748  15.829141  15.120965  15.724021  15.733311  15.369484
ENSMUSG00000021751  17.445992  17.390035  17.318620  17.900860  17.217578
ENSMUSG00000021752  10.389948  11.430448  11.201615  11.015735  11.451517
ENSMUSG00000021756  16.709190  17.193672  16.760313  16.853076  17.076717
ENSMUSG00000021758   6.047305   6.990672   6.652227   6.870711   6.780761
ENSMUSG00000021759  10.203513  12.9928

ENSMUSG00000022098  13.560635  11.305518  11.993033  11.373483  10.960087
ENSMUSG00000022099   7.541844   7.701055   7.489821   7.736461   7.643274
ENSMUSG00000022101   6.047305   6.216307   6.060645   6.052669   6.034162
ENSMUSG00000022102   9.361593   9.078499   9.384680   9.723589   9.297375
ENSMUSG00000022103   7.111306   7.457189   6.687417   6.860180   6.918829
ENSMUSG00000022105  11.551715  11.618377  11.268555  11.684348  12.035865
ENSMUSG00000022109  10.463240  10.657980  10.927868  10.854205  10.816080
ENSMUSG00000022110  15.285721  14.672916  14.754507  14.941203  14.680405
ENSMUSG00000022111  10.897065  10.195304  10.426579  10.481011  10.486476
ENSMUSG00000022112   6.047305   6.005025   6.060645   6.052669   6.034162
ENSMUSG00000022114   8.267617   8.589920   8.439413   8.714838   8.547462
ENSMUSG00000022122  12.649134  15.574710  14.014570  15.370649  15.261900
ENSMUSG00000022124  13.308006  14.211451  13.129261  13.577091  13.718773
ENSMUSG00000022129   6.047305   6.0050

ENSMUSG00000022433  10.642218  11.649666  11.854670  11.665636  11.752230
ENSMUSG00000022436   8.744501   9.356720   9.519611  10.272038   9.345092
ENSMUSG00000022437  14.715936  15.782131  15.028468  15.165283  14.931737
ENSMUSG00000022438   9.248283  10.684462  10.718462  10.964490  10.303326
ENSMUSG00000022442   8.977044   8.831811   8.690640   9.001244   9.228390
ENSMUSG00000022443  16.765146  16.952461  16.635493  16.781425  16.750845
ENSMUSG00000022449   6.047305   6.005025   6.060645   6.052669   6.586413
ENSMUSG00000022450  14.702682  14.696940  15.857442  15.574085  14.705319
ENSMUSG00000022451  14.530059  14.342292  13.443869  13.879651  14.038560
ENSMUSG00000022452  13.492284  12.920476  13.932750  14.134675  13.476768
ENSMUSG00000022454   6.047305   6.005025   6.060645   6.052669   6.034162
ENSMUSG00000022462  15.786066  16.908506  16.201227  16.923853  16.477497
ENSMUSG00000022463  14.029214  11.570293  12.188700  12.292115  12.288416
ENSMUSG00000022464  19.472704  19.1988

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

[1] 2


In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_course_30.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
[1]  8  9 37 40

$Fold02
[1]  1 10 13 35

$Fold03
[1]  2 21 27 28

$Fold04
[1] 11 12 22 29

$Fold05
[1] 16 18 19 36

$Fold06
[1] 15 25 31 38

$Fold07
[1]  5  6 23 26

$Fold08
[1]  7 30 33 34

$Fold09
[1] 17 24 32 39

$Fold10
[1]  3  4 14 20



In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

[1]  0 30


In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_course_30.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_course_30.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_course_30.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951106 371.3   13077480 698.5 10918844 583.2
Vcells 13996630 106.8   25842636 197.2 25842636 197.2


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_course_30.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_course_30.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

[1]  8  9 37 40
[1] "----------"
[1]  1 10 13 35
[1] "----------"
[1]  2 21 27 28
[1] "----------"
[1] 11 12 22 29
[1] "----------"
[1] 16 18 19 36
[1] "----------"
[1] 15 25 31 38
[1] "----------"
[1]  5  6 23 26
[1] "----------"
[1]  7 30 33 34
[1] "----------"
[1] 17 24 32 39
[1] "----------"
[1]  3  4 14 20
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  NaN : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  NaN : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  NaN : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.2 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.0333333333333333 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last ARI =  NaN : Last ECOUNT =  19 : Last

[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  NaN : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  NaN : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  NaN : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9677051 : Last ARI =  NaN : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5336403 : Last ARI =  NaN : Last ECOUNT =  72 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-72671 : Last ARI =  NaN : Last ECOUNT =  90 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94 % remaining..."
[1] "Last RXN_ID =  R-MMU-9633741 : Last ARI =  NaN : Last ECOUNT =  85 : Last Lung MISCLASS 

[1] "Last RXN_ID =  R-MMU-927830 : Last ARI =  NaN : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-448955 : Last ARI =  NaN : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89 % remaining..."
[1] "Last RXN_ID =  R-MMU-200652 : Last ARI =  -0.15 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9624798 : Last ARI =  0.266666666666667 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-2213248 : Last ARI =  NaN : Last ECOUNT =  64 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5618080 : Last ARI =  NaN : Last ECOUNT =  40 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852298 : Last ARI =  NaN : Last ECOUNT =  17 : Last L

[1] "Last RXN_ID =  R-MMU-1363303 : Last ARI =  NaN : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-188386 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-390593 : Last ARI =  NaN : Last ECOUNT =  35 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-264458 : Last ARI =  NaN : Last ECOUNT =  50 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-5635854 : Last ARI =  NaN : Last ECOUNT =  52 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-68948 : Last ARI =  NaN : Last ECOUNT =  50 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8939801 : Last ARI =  NaN : Last ECOUNT =  47 : Last Lung MISCLASS = 

[1] "Last RXN_ID =  R-MMU-182990 : Last ARI =  NaN : Last ECOUNT =  21 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-448421 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8950405 : Last ARI =  0.233333333333333 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-8983835 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78 % remaining..."
[1] "Last RXN_ID =  R-MMU-8984012 : Last ARI =  NaN : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5666160 : Last ARI =  NaN : Last ECOUNT =  94 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-201685 : Last ARI =  NaN : Last ECOUNT =  26 : Last Lung

...

[1] "Last RXN_ID =  R-MMU-9623384 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  24.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9007731 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  24.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-139919 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  24.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-70241 : Last ARI =  0.5 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  24 % remaining..."
[1] "Last RXN_ID =  R-MMU-1475436 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9706323 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-5607750 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : L

[1] "Last RXN_ID =  R-MMU-5683792 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-186509 : Last ARI =  0.133333333333333 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9655549 : Last ARI =  0.366666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9756138 : Last ARI =  NaN : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2161293 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9644315 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-5696131 : Last ARI =  NaN : Last ECOUNT =

[1] "Last RXN_ID =  R-MMU-1483002 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-429961 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-264689 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8858252 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-189406 : Last ARI =  0.466666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-6784734 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9654054 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung 

[1] "Last RXN_ID =  R-MMU-6814086 : Last ARI =  0.366666666666667 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8849068 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-9692867 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-8986307 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8 % remaining..."
[1] "Last RXN_ID =  R-NUL-844613 : Last ARI =  -0.0333333333333333 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9629360 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-6809624 : Last ARI =  0.6 : Last ECOUNT =  1 : 

[1] "Last RXN_ID =  R-MMU-9603983 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-111883 : Last ARI =  0.366666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9622513 : Last ARI =  -0.1 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5661125 : Last ARI =  NaN : Last ECOUNT =  14 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9695949 : Last ARI =  0.466666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1855172 : Last ARI =  0.116666666666667 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6807055 : Last ARI =  0.6666666

In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_course_30.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_course_30.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_course_30.Rds", sep = ""))

In [28]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_course_30.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0           NaN     14
R-MMU-170666.30.00   R-MMU-170671        0           NaN     14
R-MMU-170671.0.00    R-MMU-170674        0           NaN     14
R-MMU-170671.30.00   R-MMU-170686        0           NaN     14
R-MMU-170674.0.00    R-MMU-380073        0           NaN    182
R-MMU-170674.30.00   R-MMU-392187        0           NaN     22
R-MMU-170686.0.00    R-MMU-392195        0           NaN     22
R-MMU-170686.30.00   R-MMU-392202        0           NaN     22
R-MMU-380073.0.00    R-MMU-749454        0           NaN    178
R-MMU-380073.30.00   R-MMU-749456        0           NaN    178
R-MMU-392187.0.00    R-MMU-751001        0           NaN     21
R-MMU-392187.30.00  R-MMU-8964229        0           NaN      5
R-MMU-392195.0.00   R-MMU-8964242        0           NaN      5
R-MMU-392195.30.00  R-MMU-8964329        0           NaN      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-205099.30.00  R-MMU-5654224        0           NaN     12
R-MMU-205112.0.00   R-MMU-5654392        0           NaN     20
R-MMU-205112.30.00  R-MMU-5654397        0           NaN     16
R-MMU-205117.0.00   R-MMU-5654399        0           NaN     16
R-MMU-205117.30.00  R-MMU-5654402        0           NaN     20
R-MMU-209532.0.00   R-MMU-5654404        0           NaN     16
R-MMU-209532.30.00  R-MMU-5654406        0           NaN     18
R-MMU-209566.0.00   R-MMU-5654407        0           NaN     16
R-MMU-209566.30.00  R-MMU-5654408        0           NaN     12
R-MMU-9012761.0.00  R-MMU-5654409        0           NaN     12
R-MMU-9012761.30.00 R-MMU-5654413        0           NaN     17
R-MMU-3247837.0.00  R-MMU-5654416        0           NaN     15
R-MMU-3247837.30.00 R-MMU-5654418        0           NaN     14
R-MMU-3247840.0.00  R-MMU-5654422        0           NaN     14
R-MMU-3247840.30.00 R-MMU-5654423        0           NaN     16
R-MMU-3247843.0.00  R-MMU-5654425       

R-MMU-5654224.30.00 R-MMU-9011082        0           NaN      2
R-MMU-5654392.0.00  R-MMU-9011144        0  3.333333e-02      3
R-MMU-5654392.30.00 R-MMU-9011146        0  3.333333e-02      3
R-MMU-5654397.0.00  R-MMU-9753028        0           NaN      5
R-MMU-5654397.30.00 R-MMU-9753126        0           NaN      5
R-MMU-5654399.0.00   R-MMU-514604        0  3.666667e-01      2
R-MMU-5654399.30.00  R-MMU-975389        0           NaN      4
R-MMU-5654402.0.00  R-MMU-1169394        0           NaN      5
R-MMU-5654402.30.00 R-MMU-1169398        0           NaN      5
R-MMU-5654404.0.00  R-MMU-1169402        0           NaN      6
R-MMU-5654404.30.00 R-MMU-1169403        0           NaN      4
R-MMU-5654406.0.00  R-MMU-1169405        0           NaN      5
R-MMU-5654406.30.00 R-MMU-1169406        0           NaN     12
R-MMU-5654407.0.00  R-MMU-1678842        0           NaN     13
R-MMU-5654407.30.00 R-MMU-1678843        0           NaN      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-5654659.30.00 R-MMU-2424487        0           NaN     22
R-MMU-5654662.0.00   R-MMU-388814        0  3.666667e-01      6
R-MMU-5654662.30.00  R-MMU-388829        0           NaN      9
R-MMU-5654663.0.00   R-MMU-388831        0           NaN      8
R-MMU-5654663.30.00  R-MMU-388832        0           NaN      9
R-MMU-5654664.0.00   R-MMU-388833        0           NaN     17
R-MMU-5654664.30.00  R-MMU-389158        0           NaN      9
R-MMU-5654667.0.00   R-MMU-389348        0           NaN      8
R-MMU-5654667.30.00  R-MMU-389350        0           NaN      8
R-MMU-5654669.0.00   R-MMU-389352        0           NaN      7
R-MMU-5654669.30.00  R-MMU-389354        0           NaN      7
R-MMU-5654672.0.00   R-MMU-389381        0           NaN      6
R-MMU-5654672.30.00  R-MMU-389758        0           NaN     22
R-MMU-5654673.0.00   R-MMU-389759        0           NaN      7
R-MMU-5654673.30.00  R-MMU-389762        0           NaN      5
R-MMU-5654677.0.00   R-MMU-434836       

R-MMU-9011082.30.00 R-MMU-9692139        0  2.000000e-01      2
R-MMU-9011144.0.00  R-MMU-9692145        0  5.000000e-01      2
R-MMU-9011144.30.00 R-MMU-9692156        0  4.500000e-01      4
R-MMU-9011146.0.00  R-MMU-9692157        0  4.500000e-01      4
R-MMU-9011146.30.00 R-MMU-9692268        0  1.500000e-01      2
R-MMU-9753028.0.00   R-MMU-450434        0  2.166667e-01      1
R-MMU-9753028.30.00  R-MMU-450466        0           NaN     45
R-MMU-9753126.0.00   R-MMU-450551        0           NaN      7
R-MMU-9753126.30.00  R-MMU-450580        0           NaN     12
R-MMU-514604.0.00     R-MMU-72103        0           NaN     43
R-MMU-514604.30.00  R-MMU-9770119        0           NaN     69
R-MMU-975389.0.00   R-MMU-9770141        0           NaN    123
R-MMU-975389.30.00  R-MMU-9770142        0           NaN    132
R-MMU-1169394.0.00  R-MMU-9770145        0           NaN    145
R-MMU-1169394.30.00 R-MMU-9770236        0           NaN    119
R-MMU-1169398.0.00  R-MMU-6798748       

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 R-MMU-9688831        0           NaN      7
R-MMU-9692296.0.00  R-MMU-9701000        0           NaN      8
R-MMU-9692296.30.00 R-MMU-9706354        0           NaN      9
R-MMU-8869241.0.00  R-MMU-9708517        0           NaN     11
R-MMU-8869241.30.00 R-MMU-9750942        0           NaN      7
R-MMU-8866268.0.00  R-MMU-9750946        0           NaN     10
R-MMU-8866268.30.00  R-MMU-975097        0           NaN     11
R-MMU-9638046.0.00   R-MMU-975103        0           NaN     11
R-MMU-9638046.30.00  R-MMU-975122        0           NaN     12
R-MMU-1675773.0.00   R-MMU-975147        0           NaN      7
R-MMU-1675773.30.00 R-MMU-9755304        0           NaN      7
R-MMU-1675810.0.00  R-MMU-9759154        0           NaN      7
R-MMU-1675810.30.00 R-MMU-9762091        0           NaN      9
R-MMU-1676082.0.00  R-MMU-9793444        0           NaN     10
R-MMU-1676082.30.00 R-MMU-9793451        0           NaN     10
R-MMU-1676134.0.00  R-MMU-9793472        0           NaN   

R-MMU-5638006.30.00 R-MMU-4084980        0           NaN      6
R-MMU-5638007.0.00  R-MMU-6797913        0           NaN      1
R-MMU-5638007.30.00  R-MMU-442289        0           NaN      2
R-MMU-5638016.0.00  R-MMU-5218820        0           NaN      8
R-MMU-5638016.30.00 R-MMU-9013023        0           NaN     19
R-MMU-141186.0.00   R-MMU-9014433        0           NaN     13
R-MMU-141186.30.00  R-MMU-9018804        0           NaN      6
R-MMU-374909.0.00   R-MMU-8949178        0  7.000000e-01      6
R-MMU-374909.30.00  R-MMU-8953461        0  7.000000e-01      6
R-MMU-379382.0.00   R-MMU-9014295        0           NaN     18
R-MMU-379382.30.00  R-MMU-5577234        0           NaN      7
R-MMU-379395.0.00   R-MMU-3238687        0           NaN      2
R-MMU-379395.30.00  R-MMU-3238692        0           NaN      8
R-MMU-9677917.0.00  R-MMU-4641249        0           NaN      9
R-MMU-9677917.30.00 R-MMU-5323545        0  2.666667e-01      4
R-MMU-8948139.0.00  R-MMU-9793405       

R-MMU-9833004.30.00 R-MMU-9605700        0  1.333333e-01      1
R-MMU-9010544.0.00   R-MMU-112437        0  2.666667e-01      4
R-MMU-9010544.30.00 R-MMU-9603580        0  3.333333e-02      2
R-MMU-9010548.0.00  R-MMU-9603585        0  3.333333e-02      2
R-MMU-9010548.30.00 R-NUL-9603625        0  3.333333e-02      2
R-MMU-350769.0.00   R-MMU-5362522        0  8.000000e-01      3
R-MMU-350769.30.00  R-MMU-9830411        0           NaN      4
R-MMU-5635839.0.00  R-MMU-6788523        0           NaN      5
R-MMU-5635839.30.00   R-MMU-73571        0           NaN      1
R-NUL-5610714.0.00    R-MMU-73573        0           NaN      1
R-NUL-5610714.30.00   R-MMU-73577        0           NaN      1
R-NUL-5610715.0.00  R-MMU-1676149        0           NaN      1
R-NUL-5610715.30.00 R-MMU-1855205        0           NaN      1
R-NUL-5610751.0.00   R-MMU-199456        0           NaN      1
R-NUL-5610751.30.00 R-MMU-6807105        0           NaN      1
R-MMU-5696021.0.00  R-MMU-6807126       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_course_30.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_course_30.txt",sep=""), header=FALSE)
print(sid)

In [30]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  # 绘制点，指定点的大小
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  # 添加标签
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  # 手动指定颜色
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  # 将 Section 转换为因子
     colors = c("red", "green"),  # 定义颜色
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  # 在悬停文本中显示 Project ID
)
saveWidget(p, "plotly_chart030_time_course_30.html", selfcontained = TRUE)